In [ ]:
from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag
import requests
import re

url = 'https://www.aonprd.com/'
href = 'MagicWondrous.aspx?FinalSlot=Belts'


class ArchivesNethysScraper:
    def __init__(self, url, href):
        self.url = url
        self.href = href
        self.page = requests.get(url+href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")

    def table_rows(self):
        for tr in self.table.find_all('tr')[1:]:
            yield tr

    def get_item_info(self, table_row):
        output = {}
        tag = table_row.td
        href = tag.a.attrs['href']
        page = requests.get(url+href)
        attributes = ['Slot', 'Price', 'Weight']
        
        def fetch_info(left_tag, right_tag, format_output):
            regex = re.compile(
                f'{left_tag}(.*){right_tag}'
            )
            return format_output(regex.findall(page.text)[0])
        format_string = lambda str: str.replace('\r', '').replace(';', '')
        for i in range(len(attributes)-1):
            l = f'<b>{attributes[i]}</b>'
            r = f'<b>{attributes[i+1]}</b>'
            output[attributes[i].lower()] = fetch_info(l, r, format_string)
        l = r
        r = '<h3 class="framing">Description</h3>'
        output[attributes[2].lower()] = fetch_info(l, r,format_string)
        output['description'] = fetch_info(
            '<h3 class="framing">Description</h3>',
            '<h3 class="framing">Construction</h3>',
            lambda str: str.replace('\r', '')
        )
        return output
  
belts = ArchivesNethysScraper(url, href)
for tr in belts.table_rows():
    print(belts.get_item_info(tr))

# body = ArchivesNethysScraper(url, 'MagicWondrous.aspx?FinalSlot=Body')
# for tr in body.table_rows():
#    print(body.get_item_info(tr))
#    break